### 散列 Hashing

通过对数据项所处位置的先验知识，使得查找次数降低到常数级别

### 散列表 hash table

又称哈希表，是一种数据集，其中数据项的存储方式有利于快速定位

散列表中的每一个存储位置，称为槽 slot，可以用来保存数据项，且有唯一的名称

### 散列函数 hash function

用于实现数据项到存储槽名称的转换

**完美散列函数**

如果一个散列函数能把每个数据项映射到不同槽中，那么这个散列函数就可以称为完美散列函数，但现实中很难有这种情况，不同的数据项经过散列函数，映射到同一个槽中的现象称为**冲突**

好的散列函数应该具备的特性：
- 冲突少（近似完美）
- 计算难度低（额外开销少）
- 充分分散数据项（节约空间）

### 散列函数的设计
**折叠法**

将数据项按照位数分为若干段，再将几段数字想加，然后求余得到散列值

有时候会增加隔数反转的步骤，比如(62,76,54,37)变为(62,67,54,73)，虽然理论上看毫无必要，但实际作为一种微调手段，可以使结果更好符合散列特性

**平方取中法**

首先将数据项做平方运算，取平方数的中间两位，再对散列表的大小求余（例如44，平方1936，取93，对11求余得5）

**非数项**

通过ASCII转码转化成数字项，相加求和，可以后续求余
```
def hash(string, tablesize):
    sum = 0
    for pos in range(len(string)):
        sum = sum + ord(astring[pos])

    return sum % tablesize
```

对于**变位词**情况（比如item和time，通过上面的hash函数会得到同样解），可以使用字符串所在位置作为权重因子，乘以其ascii码
```
def hash_anagram(string, tablesize):
    sum = 0
    for pos in range(len(string)):
        sum = sum + pos * ord(astring[pos])

    return sum % tablesize
```


### 冲突解决

**开放定址 open addressing**

如何解决
- 向后按照一定的间隔寻找空槽放置数据项，这个过程又可以称作**再散列 rehashing**
- 如果到尾部，就回到首部继续
- 直到回到最初的槽，说明没有空槽可以插入
- 如果按照逐个槽寻找，称作**线性探测 linear probing**
- 如果不是逐个查找，需要注意跳跃的距离取值，不能被散列表大小整除，否则会产生周期，造成很多空槽永远无法被探测到
- 对于散列表大小的设计，建议设计为素数，如11,13,17等
- 还可以将线性探测变为**二次探测 quadratic probing**，逐步增加跳跃值（1,3,5,7,9...)，这样槽号就会以平方数增加（h+1,h+4,h+9...）

如何查找
- 如果在散列位置没有找到查找项，就必须做顺序查找
- 直到找到查找项，或碰到空槽（查找失败）

方案缺点
- 线性探测法，如果一个槽冲突的数据项较多，这些数据项就会在槽附近聚集，还会影响到其他数据项插入
---
**数据项链 chaining**

如何解决
- 将容纳单个数据项的槽扩展为容纳数据项集合
- 散列冲突发生时，只需要简单的将数据线添加到数据项集合中

如何查找
- 需要查找同一个槽中的整个集合


### 抽象数据实现

**ADT Map的功能**
- 保存key-data键值对
- 使用关键码key可以查询数据值data
- 这种键值关联的方法称为**映射Map**
- 关键码具有唯一性
- 通过关键码可以唯一确定一个数据值

**Map操作**

- Map()：创建一个空映射，返回空映射对象
- put(key, val)：将key-val关联对加入映射中，如果key已存在，将val替换旧关联值
- get(key)：给定key，返回关联的数据值，如果存在，返回None
- del：通过del map\[key\]的语句形式删除key-val关联
- len()：返回映射中key-val关联的数目
- in：通过key in map的语句形式，返回key是否存在于关联中，布尔值

**Map实现**
- 包含两个列表作为成员
- slot列表用于保存key
- data列表用户保存数据项
- 在slot列表中找到key的位置后，在data列表的对应**相同位置**的数据项即为关联数据

In [1]:
#冲突采用线性探测
class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size

    def hashfunction(self, key):
        return key % self.size

    def  rehash(self, oldhash):
        return (oldhash + 1) % self.size

    def put(self, key, data):
        hashvalue = self.hashfunction(key)

        if self.slots[hashvalue] == None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data #更新替换
            else:
                nextslot = self.rehas(hashvalue)
                while self.slots[nextslot] != None and self.slots[nextslot] != key and nextslot != hashvalue:
                    nextslot = self.rehash(nextslot)

                if self.slots[nextslot] == None:
                    self.slots[nextslot] = key
                    self.data[nextslot] = data
                elif nextslot == hashvalue:
                    return None
                else:
                    self.data[nextslot] = data

    def get(self, key):
        startslot = self.hashfunction(key)

        data = None
        stop = False
        found = False
        position = startslot
        while self.slots[position] !=None and not found and not stop:
            if self.slots[position] == key:
                found = True
                data = self.data[position]
            else:
                position = self.rehash(position)
                if position == startslot:
                    stop = True
        return data

        #实现[]访问
        def __getitem__(self, key):
            return self.get(key)

        def __setitem__(self, key, data):
            self.put(key, data)

### 散列算法分析
散列在最好情况下，可以提供O(1)常数级时间复杂度的查找性能

但由于存在冲突，因此查找次数并没有那么简单

评估散列冲突的最重要信息就是负载因子$\lambda$，一般来说：
- 如果$\lambda$较小，散列冲突的几率就小，数据项通常会保存在所属的散列槽中
- 如果$\lambda$较大，以为着散列表填充较满，冲突会越来越多；如果采用数据链的话，意味着每条链上的数据项增多

如果用线性探测的开放定址法解决冲突：
- 成功查找，平均需要比对次数为$\frac{1}{2}(1+\frac{1}{1-\lambda})$
- 不成功查找，平均比对次数为$\frac{1}{2}(1+(\frac{1}{1-\lambda})^2)$

如果用数据链解决冲突：
- 成功查找，平均需要比对次数为$1+\frac{\lambda}{2}$
- 不成功查找，平均比对次数为$\lambda$